In [11]:
pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 460kB 36.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import sys
%cd /content/drive/My Drive/Colab Notebooks/news-comments_emotion_classification-CNN-tensorflow-master/util/
!ls

/content/drive/My Drive/Colab Notebooks/news-comments_emotion_classification-CNN-tensorflow-master/util
load_data.py  __pycache__  savefig_500dpi.png  text_preprocessing.py


In [13]:
sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/news-comments_emotion_classification-CNN-tensorflow-master/util/')

In [14]:
import numpy as np
import pandas as pd
import os
import tqdm
import itertools
from collections import Counter

import sys
sys.path.append('../')

from util.text_preprocessing import tokenize_okt

# Load Data

## 전체 코멘트 불러오기

In [5]:
total_comments_df = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/news-comments_emotion_classification-CNN-tensorflow-master/dataset/total_raw_token/0710 comments_with_okt-tokenized.pkl")

In [6]:
total_comments_df

,Unnamed: 0,Sentence,comment_token
0,0,청소 네가 대신 해 줘,"[청소, 네, 대신, 해, 주다]"
1,1,둘 다 청소 하기 싫어 귀찮아,"[둘, 청소, 싫다, 귀찮다]"
2,2,둘 다 하기 싫어서 화내,"[둘, 싫다, 화내]"
3,3,그럼 방세는 어떡해,"[그렇다, 방, 세다, 어떻다, 해]"
4,4,권택인 줄 알았는데 그런 사람이 생겼나 보더라고,"[권, 택인, 줄, 알다, 그렇다, 사람, 생기다, 보더]"
...,...,...,...
62512,62512,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,"[솔직하다, 예보, 제대로, 데, 세금, 아끼다, 그냥, 폐지]"
62513,62513,재미가 없으니 망하지,"[재미, 없다, 망하다]"
62514,62514,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,"[공장, 도시락, 비우다, 생, 적임, 아르바이트, 화장실, 가성, 손, 않씯, 재..."
62515,62515,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,"[코딱지, 만, 나라, 지다, 들다, 끼리, 피터지다, 싸우다, 세다, 징, 클래스]"


In [15]:
total_comments_df = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/news-comments_emotion_classification-CNN-tensorflow-master/dataset/total_raw_token/0710 comments_with_okt-tokenized.pkl")
total_comments_df = total_comments_df.loc[:,['Sentence','comment_token']]
total_comments_df.head(5)

,Sentence,comment_token
0,청소 네가 대신 해 줘,"[청소, 네, 대신, 해, 주다]"
1,둘 다 청소 하기 싫어 귀찮아,"[둘, 청소, 싫다, 귀찮다]"
2,둘 다 하기 싫어서 화내,"[둘, 싫다, 화내]"
3,그럼 방세는 어떡해,"[그렇다, 방, 세다, 어떻다, 해]"
4,권택인 줄 알았는데 그런 사람이 생겼나 보더라고,"[권, 택인, 줄, 알다, 그렇다, 사람, 생기다, 보더]"


## 감정 사전 만들기
- 6 emotion : happy, sad, disgust, angry, surprised, fear
- 홍종선, 정연주. (2009). 감정동사의 범주 규정과 유형 분류. 한국어학, 45(), 387-420.

In [16]:
six_emodict_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/news-comments_emotion_classification-CNN-tensorflow-master/자료/six_emotion_bigram2_0403_1.csv", encoding="CP949")
six_emodict_df.head()

,happy,sad,disgust,angry,surprised,fear
0,가뿐하다,가슴앓이,가소롭다,갈기갈기,갑작스럽다,가혹하다
1,감개무량하다,가엽다,거북하다,개새끼,경악하다,강압적
2,감격스럽다,가엾다,경박하다,개자식,경이,겁쟁이
3,감격하다,가혹하다,괴상하다,격노하다,급작스럽다,공포감
4,감동스럽다,각박하다,괴팍하다,격분하다,기겁하다,공포스럽다


In [9]:
# 감정 별 단어 개수
six_emodict_df.count()

happy        113
sad          174
disgust       77
angry        110
surprised     47
fear          38
dtype: int64

In [42]:
# 감정사전 리스트를 토큰화, nan값 제거, 중복 제거한 뒤, 각 감정변수에 차례대로 할당함.
# 감정 리스트에 변수 할당 
six_emotion = six_emodict_df.columns.tolist()
print(six_emotion)

for emo in six_emotion:
    # 단어 사전에서 emo_words로 받아오기
    emo_words = six_emodict_df[emo].dropna().tolist() 
    # 단어 토큰화, 토큰화 후 nan값 된 단어 제거(len(tokenize_okt(x))>0)
    emo_words = [tokenize_okt(x)[0] for x in emo_words if len(tokenize_okt(x))>0]
    # 중복 제거(set)
    emo_words = sorted(list(set(emo_words)))
    # 각 감정변수에 차례대로 할당
    globals()[emo] = emo_words
    # globals()함수: 현재 global 변수들을 딕셔너리 형태로 return (ex: {'emo':emo_words})

angry[:5]


['happy', 'sad', 'disgust', 'angry', 'surprised', 'fear']


['가당찮다', '가증', '갈기갈기', '갈등', '강압']

In [40]:
bi_happy = [('속','시원하다'),('낯','간지럽다'),('정','들다'),('홀','가분')]

bi_sad = [('가슴','아프다'),('기운','없다'),('남','부럽다'),('뒤','숭숭'),('맥','빠지다'),
              ('복','받치다'),('뼈','아프다'),('뼈','저리다'),('애','끓다'),('애','닮다'),
              ('애','타다'),('처','연하다'),('속','썩다'),('속','앓이'),('속','타다')]

bi_disgust =  [('못','밉다'),('정','떨어지다'),('못','마땅하다'),('시','덥다'),('남','부끄럽다'),
                   ('징','그렇다'),('뼈','저리다'),('애','끓다'),('애','닮다'),('처','연하다'),
                   ('속','썩다'),('속','앓이'),('속','타다')]

bi_angry = [('못','되다'),('삐','딱하다'),('비','딱하다'),('약','오르다'),('치','떨리다'),
                ('치밀','오르다'),('성','나다'),('못','마땅하다'),('정','떨어지다'),('남','부끄럽다')]

bi_surprised = [('기상','천외'),('얼','떨다')]

bi_fear = [('뒤','숭숭')]

# 코멘트 데이터에 감정 레이블 달기 

In [41]:
test = ["가슴 아프다 정말", #- sad
        "요즘 상황이 뒤숭숭하다",#sad fear
        "어어어어엉 우웅우 감동스럽다",#happy
       "낯간지러워"]
        
test_token = []
for i in test:
    test_token.append(tokenize_okt(i))
print(test_token)

[['가슴', '아프다', '정말'], ['요즘', '상황', '뒤', '숭숭'], ['어어', '엉', '우웅', '우', '감동', '스럽다'], ['낯', '간지럽다']]


## 댓글에 특정 감정에 해당하는 단어가 등장하면 update

- bigram
- unigram

In [43]:
total_comments_df['comment_token'][:10]

0                   [청소, 네, 대신, 해, 주다]
1                     [둘, 청소, 싫다, 귀찮다]
2                          [둘, 싫다, 화내]
3                 [그렇다, 방, 세다, 어떻다, 해]
4     [권, 택인, 줄, 알다, 그렇다, 사람, 생기다, 보더]
5                         [그냥, 걷다, 있다]
6                           [고등학교, 동창]
7    [처음, 학원, 만나다, 서로, 좋아하다, 사귀다, 되어다]
8             [내, 애정, 표현, 자다, 자주, 싸우다]
9                           [오늘, 헤어지다]
Name: comment_token, dtype: object

In [67]:
emo_set = [happy, sad, disgust, angry, surprised, fear]
emo_bigram_set = [bi_happy, bi_sad, bi_disgust, bi_angry, bi_surprised, bi_fear]

check_token_lst = []
emo_cnt_lst = []

total_num = len(total_comments_df)
# N-gram: 문자열을 N개의 기준 단위로 절단하는 방법.
# comment 돌기
for i,comment in enumerate(total_comments_df['comment_token']):
    print(i/total_num,end="\r") # 1이 되면 루프 끝남.
    check_token = []
    emo_cnt = np.zeros(len(emo_set))
    
    # bigram 처리: 두 개의 단어.
    for j, biwords in enumerate(emo_bigram_set):
        for a, b in biwords:
             if (a in comment) & (b in comment):
                check_token.append(a+b)
                emo_cnt[j] += 1 

    # unigram 처리: 하나의 단어
    for token in comment:
        for k, words in enumerate(emo_set):
            if token in words:
                emo_cnt[k] += 1
                check_token.append(token)

    check_token_lst.append(check_token)
    emo_cnt_lst.append(emo_cnt)

In [70]:
# 비율로 환산
# .sum(axis=1,keepdims=True):: 서로 다른 열별(가로) 합, 차원의 축소 방지
sum_of_emo = np.array(emo_cnt_lst).sum(axis=1,keepdims=True)

emo_ratio = np.array(emo_cnt_lst) / sum_of_emo
# 위의 나누기에서 nan이 나올 경우, 0으로 치환
emo_ratio[np.isnan(emo_ratio)] = 0

emo_df = pd.DataFrame(emo_ratio,columns = ['happy','sad','disgust','angry','surprised','fear'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [78]:
comment_with_emo = pd.concat([emo_df,
                              total_comments_df,
                              pd.DataFrame({"check":pd.Series(check_token_lst)})],
                             sort=False,axis=1)
# 최소 1개의 감정단어 등장한 댓글만 뽑기
comment_with_emo = comment_with_emo[comment_with_emo['check'].apply(len) > 0]
print(comment_with_emo.shape)
print("{}개의 댓글 중 {} 개의 댓글이 감정 단어를 보유.".format(len(total_comments_df),len(comment_with_emo)))
comment_with_emo.head()

(11814, 9)
62517개의 댓글 중 11814 개의 댓글이 감정 단어를 보유.


,happy,sad,disgust,angry,surprised,fear,Sentence,comment_token,check
0,0.0,0.0,0.0,1.0,0.0,0.0,청소 네가 대신 해 줘,"[청소, 네, 대신, 해, 주다]",[네]
1,0.0,0.0,0.5,0.5,0.0,0.0,둘 다 청소 하기 싫어 귀찮아,"[둘, 청소, 싫다, 귀찮다]","[싫다, 싫다]"
2,0.0,0.0,0.5,0.5,0.0,0.0,둘 다 하기 싫어서 화내,"[둘, 싫다, 화내]","[싫다, 싫다]"
13,0.0,0.0,0.0,0.0,0.0,1.0,부모님한테 아직 말 안 했는데 말하기가 두려워,"[부모님, 아직, 말, 안, 말, 두렵다]",[두렵다]
17,0.0,1.0,0.0,0.0,0.0,0.0,부모님도 다 슬퍼하지,"[부모님, 슬프다]",[슬프다]


In [49]:
print("{}개의 댓글 중 {} 개의 댓글이 감정 단어를 보유.".format(len(total_comments_df),len(comment_with_emo)))

62517개의 댓글 중 11814 개의 댓글이 감정 단어를 보유.


In [50]:
comment_with_emo['check'].apply(len).max()

75

In [51]:
comment_with_emo[comment_with_emo['check'].apply(len)==109]

,happy,sad,disgust,angry,surprised,fear,Sentence,comment_token,check


## 감정 단어 보유 비율에 따라 labeling

In [72]:
def label_with_major_emotion(df, ratio):
    over_index = np.where((df.happy > ratio) |
                          (df.sad > ratio) |
                          (df.disgust > ratio) |
                          (df.angry > ratio) |
                          (df.surprised > ratio) |
                          (df.fear > ratio))[0]

    over_df = df.iloc[over_index.tolist(),:]

    # Skipna = True will skip all the Na values 
    # find maximum along column axis 
    over_df['emotion'] = over_df.iloc[:,:6].idxmax(axis = 1, skipna = True)
    print(Counter(over_df.emotion))
    
    return over_df

In [73]:
label_with_major_emotion(comment_with_emo, 0.5).to_pickle("/content/drive/My Drive/Colab Notebooks/news-comments_emotion_classification-CNN-tensorflow-master/dataset/model/0710 comment_with_emo_over0.5_1.pkl")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Counter({'sad': 3176, 'happy': 2741, 'angry': 2004, 'fear': 910, 'surprised': 595, 'disgust': 505})


In [75]:
label_with_major_emotion(comment_with_emo, 0.5)

Counter({'sad': 3176, 'happy': 2741, 'angry': 2004, 'fear': 910, 'surprised': 595, 'disgust': 505})


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,happy,sad,disgust,angry,surprised,fear,Sentence,comment_token,check,emotion
0,0.0,0.0,0.000000,1.000000,0.0,0.0,청소 네가 대신 해 줘,"[청소, 네, 대신, 해, 주다]",[네],angry
13,0.0,0.0,0.000000,0.000000,0.0,1.0,부모님한테 아직 말 안 했는데 말하기가 두려워,"[부모님, 아직, 말, 안, 말, 두렵다]",[두렵다],fear
17,0.0,1.0,0.000000,0.000000,0.0,0.0,부모님도 다 슬퍼하지,"[부모님, 슬프다]",[슬프다],sad
18,1.0,0.0,0.000000,0.000000,0.0,0.0,그래 고마워,"[그렇다, 고맙다]",[고맙다],happy
37,1.0,0.0,0.000000,0.000000,0.0,0.0,이제 2달 지났어,"[이제, 달, 지나다]",[달],happy
...,...,...,...,...,...,...,...,...,...,...
62490,0.0,0.0,0.333333,0.666667,0.0,0.0,이백만이라고 해라나도 박근혜 싫지만언론이 앞장서서 이런 쓰레기,"[이백만이, 해, 라나, 박근혜, 싫다, 언론, 앞장서다, 이렇다, 쓰레기]","[싫다, 싫다, 쓰레기]",angry
62492,0.0,0.0,0.000000,1.000000,0.0,0.0,위증교사까지하는 쓰레기 짓거리를 서슴없이 하다니,"[위증, 교사, 쓰레기, 짓거리, 서슴없이]",[쓰레기],angry
62493,0.0,0.0,0.000000,1.000000,0.0,0.0,돼지새끼 빨리 죽어라 지옥가게,"[돼지, 새끼, 빨리, 죽다, 지옥, 가게]",[새끼],angry
62508,1.0,0.0,0.000000,0.000000,0.0,0.0,트럼프는 박그네한테 고마워할듯? ㅋㅋ,"[트럼프, 박, 그네, 고맙다]",[고맙다],happy
